In [1]:
# 라이브러리 및 모듈 import
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import cv2
import os
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet
import pandas as pd
from tqdm import tqdm
from glob import glob

In [2]:
# CustomDataset class 선언

class CustomDataset(Dataset):
    '''
      data_dir: data가 존재하는 폴더 경로
      transforms: data transform (resize, crop, Totensor, etc,,,)
    '''

    def __init__(self, annotation, data_dir, transforms=None):
        super().__init__()
        self.data_dir = data_dir
        # coco annotation 불러오기 (coco API)
        self.coco = COCO(annotation)
        self.predictions = {
            "images": self.coco.dataset["images"].copy(),
            "categories": self.coco.dataset["categories"].copy(),
            "annotations": None
        }
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.coco.getImgIds(imgIds=index)

        image_info = self.coco.loadImgs(image_id)[0]
        
        image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        # 라벨 등 이미지 외 다른 정보 없기 때문에 train dataset과 달리 이미지만 전처리
        
        # transform
        if self.transforms:
            sample = self.transforms(image=image)

        return sample['image'], image_id
    
    def __len__(self) -> int:
        return len(self.coco.getImgIds())

In [3]:
# Albumentation을 이용, augmentation 선언
def get_train_transform():
    return A.Compose([
        A.Resize(512, 512),
        A.Flip(p=0.5),
        ToTensorV2(p=1.0)
    ])


def get_valid_transform():
    return A.Compose([
        A.Resize(512, 512),
        ToTensorV2(p=1.0)
    ])

In [4]:
from effdet import DetBenchPredict
import gc

# Effdet config를 통해 모델 불러오기 + ckpt load
def load_net(checkpoint_path, device):
    config = get_efficientdet_config('tf_efficientdet_d3')
    config.num_classes = 10
    config.image_size = (512,512)
    
    config.soft_nms = False
    config.max_det_per_image = 50
    
    net = EfficientDet(config, pretrained_backbone=False)
    net.class_net = HeadNet(config, num_outputs=config.num_classes)
    
    checkpoint = torch.load(checkpoint_path, map_location='cpu')

    net = DetBenchPredict(net)
    net.load_state_dict(checkpoint)
    net.eval()

    return net.to(device)

In [5]:
# valid function
def valid_fn(val_data_loader, model, device):
    outputs = []
    for images, image_ids in tqdm(val_data_loader):
        # gpu 계산을 위해 image.to(device)       
        images = torch.stack(images) # bs, ch, w, h 
        images = images.to(device).float()
        output = model(images)
        for out in output:
            outputs.append({'boxes': out.detach().cpu().numpy()[:,:4], 
                            'scores': out.detach().cpu().numpy()[:,4], 
                            'labels': out.detach().cpu().numpy()[:,-1]})
    return outputs

def collate_fn(batch):
    return tuple(zip(*batch))

In [9]:
def main():
    annotation = '../../dataset/test.json'
    data_dir = '../../dataset'
    val_dataset = CustomDataset(annotation, data_dir, get_valid_transform())
    for fold, epoch  in zip([1, 2, 3, 4, 5], [81, 46, 62, 65, 55]):
        
        checkpoint_path = f'./pretrained/fold{fold}/epoch_{epoch}.pth'
        # epoch = checkpoint_path.split("/")[-1].split("_")[-1].split(".")[0]
        print(f'fold{fold}, epoch{epoch}: {checkpoint_path}')

        score_threshold = 0.1
        val_data_loader = DataLoader(
            val_dataset,
            batch_size=4,
            shuffle=False,
            num_workers=4,
            collate_fn=collate_fn
        )

        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        print(device)

        model = load_net(checkpoint_path, device)

        outputs = valid_fn(val_data_loader, model, device)

        prediction_strings = []
        file_names = []
        coco = COCO(annotation)

        for i, output in enumerate(outputs):
            prediction_string = ''
            image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
            for box, score, label in zip(output['boxes'], output['scores'], output['labels']):
                if score > score_threshold:
                    prediction_string += str(int(label)-1) + ' ' + str(score) + ' ' + str(box[0] * 2) + ' ' + str(
                        box[1] * 2) + ' ' + str(box[2] * 2) + ' ' + str(box[3] * 2) + ' '
            prediction_strings.append(prediction_string)
            file_names.append(image_info['file_name'])

        submission = pd.DataFrame()
        submission['PredictionString'] = prediction_strings
        submission['image_id'] = file_names
        submission.to_csv(f'effdet_submissions/effdet_fold{fold}_epoch_{epoch}.csv', index=None)
        print(submission.head())

In [10]:
if __name__ == '__main__':
    main()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
fold1, epoch81: ./pretrained/fold1/epoch_81.pth
cuda


100%|██████████| 1218/1218 [03:26<00:00,  5.88it/s]


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
                                    PredictionString       image_id
0  7 0.9299216 603.4823608398438 512.719299316406...  test/0000.jpg
1  5 0.62437433 142.91941833496094 1.875686645507...  test/0001.jpg
2  1 0.7901068 279.421630859375 261.6471557617187...  test/0002.jpg
3  9 0.74314713 145.923095703125 254.927459716796...  test/0003.jpg
4  7 0.81348485 201.53683471679688 244.2016601562...  test/0004.jpg
fold2, epoch46: ./pretrained/fold2/epoch_46.pth
cuda


100%|██████████| 1218/1218 [03:26<00:00,  5.89it/s]


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
                                    PredictionString       image_id
0  7 0.9180697 220.8009033203125 57.9095153808593...  test/0000.jpg
1  5 0.6502935 139.77149963378906 3.8359985351562...  test/0001.jpg
2  1 0.69247663 287.4536437988281 330.17990112304...  test/0002.jpg
3  9 0.70538384 143.6434326171875 254.58435058593...  test/0003.jpg
4  1 0.4975834 191.9168701171875 270.29248046875 ...  test/0004.jpg
fold3, epoch62: ./pretrained/fold3/epoch_62.pth
cuda


...

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
                                    PredictionString       image_id
0  7 0.96499103 223.342529296875 49.5193939208984...  test/0000.jpg
1  5 0.70357233 137.8030548095703 0.9522399902343...  test/0001.jpg
2  1 0.7618972 315.6864318847656 308.734680175781...  test/0002.jpg
3  9 0.51350516 133.09820556640625 263.4837646484...  test/0003.jpg
4  0 0.5149405 205.68875122070312 335.8544921875 ...  test/0004.jpg
fold4, epoch65: ./pretrained/fold4/epoch_65.pth
cuda


100%|██████████| 1218/1218 [03:32<00:00,  5.73it/s]


loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
                                    PredictionString       image_id
0  7 0.9560713 612.5043334960938 530.345642089843...  test/0000.jpg
1  5 0.8271004 135.14688110351562 6.2983703613281...  test/0001.jpg
2  1 0.8122465 99.21257019042969 290.108459472656...  test/0002.jpg
3  9 0.8097313 145.3818359375 248.24847412109375 ...  test/0003.jpg
4  7 0.78980166 183.87530517578125 264.0597839355...  test/0004.jpg
fold5, epoch55: ./pretrained/fold5/epoch_55.pth
cuda


100%|██████████| 1218/1218 [03:28<00:00,  5.85it/s]


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
                                    PredictionString       image_id
0  7 0.9540538 609.06884765625 524.638916015625 9...  test/0000.jpg
1  3 0.77380335 350.5903625488281 229.86987304687...  test/0001.jpg
2  1 0.82217926 299.5276794433594 280.63452148437...  test/0002.jpg
3  9 0.83619136 133.674072265625 266.681915283203...  test/0003.jpg
4  7 0.91149044 209.179931640625 260.293701171875...  test/0004.jpg
